In [ ]:
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
%matplotlib inline

In [ ]:
import umap

In [ ]:
from sklearn import cluster

In [ ]:
most_commented_path = ''
embeddings_path = ''
validation_authors_path = ''
user_categories_path = ''

In [ ]:
def save_to_pdf(filename, ax):
    fig = ax.get_figure()
    fig.savefig(filename, bbox_inches='tight')

In [ ]:
df = pd.read_csv(most_commented_path)

In [ ]:
df['cc'] = pd.Categorical(df.category)

In [ ]:
df['code'] = df.cc.cat.codes

In [ ]:
vectors = KeyedVectors.load_word2vec_format(embeddings_path)

In [ ]:
num_classes = df['code'].nunique()

In [ ]:
num_classes

In [ ]:
reducer = umap.UMAP(n_neighbors=5)

In [ ]:
validation_authors = list(np.load(validation_authors_path))

In [ ]:
df_user_categories = pd.read_csv(user_categories_path)

In [ ]:
most_categories = df_user_categories.groupby('category')['author_id'].nunique().reset_index()
most_categories = most_categories.sort_values(by='author_id', ascending=False).head(10)['category'].tolist()
most_categories = set(most_categories)

In [ ]:
def get_relevant_categories(x):
    if x in most_categories:
        return x
    else:
        return 'other'

In [ ]:
df_user_categories['category'] = df_user_categories.category.apply(get_relevant_categories)

In [ ]:
df_user_categories = df_user_categories[df_user_categories['category'] != 'other']

In [ ]:
df_user_categories['int_cat'], label = pd.factorize(df_user_categories.category)

In [ ]:
df_user_categories.index = df_user_categories.author_id

In [ ]:
df_user_categories['int_cat'].nunique()

In [ ]:
user_cat_dict = df_user_categories['int_cat'].to_dict()


In [ ]:
catgs = df_user_categories[['category', 'int_cat']].drop_duplicates()
catgs.index = catgs.int_cat
catgs_dict = catgs.category.to_dict()

In [ ]:
validation_authors = np.array([author for author in validation_authors if author != 737638])

In [ ]:
validation_authors = list(filter(lambda x: x in user_cat_dict, validation_authors))

In [ ]:
vector_matrix = np.zeros((len(validation_authors), vectors.vector_size))

In [ ]:
for index, author in enumerate(validation_authors):
    vector_matrix[index][:] = vectors.get_vector(str(author))

In [ ]:
clusterer = cluster.KMeans(n_clusters=10, )

In [ ]:
preds = clusterer.fit_predict(vector_matrix)

In [ ]:
embedding = reducer.fit_transform(vector_matrix)

In [ ]:
len(preds)

In [ ]:
color_palette = sns.color_palette("Accent", len(np.unique(df_user_categories.int_cat)))
color = [color_palette[user_cat_dict[author]] for author in validation_authors]
vis_df = pd.DataFrame()
vis_df['x'] = embedding[:, 0]
vis_df['y'] = embedding[:, 1]
vis_df['category'] =[catgs_dict[user_cat_dict[author]] for author in validation_authors]
vis_df['color'] = color
#vis_df = vis_df[vis_df.category != 'else']

In [ ]:
sns.set()
sns.set_context('paper', font_scale=1.8)
fig, ax = plt.subplots(1, figsize=(10, 10))
ax = sns.scatterplot(x="x", y="y", hue='category',palette='Paired', marker='+', data=vis_df, alpha=0.8, legend="full", linewidth=1)
plt.gca().set_aspect('equal', 'datalim')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., frameon=False, markerscale=4.2)
handles,labels = ax.get_legend_handles_labels()
save_to_pdf('./plots/umap_neighbor_4_node2vec.pdf', ax)

In [ ]:
fig_legend = plt.figure(figsize=(2,2))
axi = fig_legend.add_subplot(111)     
fig_legend.legend(handles[1:], labels[1:], markerscale=4.2,  ncol=5, frameon=False,)
axi.xaxis.set_visible(False)
axi.yaxis.set_visible(False)
axi.patch.set_alpha(0)
fig_legend.canvas.draw()
fig_legend.show()
save_to_pdf('./plots/legend.pdf', axi)

In [ ]:
color_palette = sns.color_palette("Accent", len(np.unique(10)))
vis_df = pd.DataFrame()
vis_df['x'] = embedding[:, 0]
vis_df['y'] = embedding[:, 1]
vis_df['category'] = preds
vis_df['color'] = color

In [ ]:
sns.set()
fig, ax = plt.subplots(1, figsize=(10, 10))
ax = sns.scatterplot(x="x", y="y", hue='category', marker='+', data=vis_df, alpha=0.8, legend=False, linewidth=1)
plt.gca().set_aspect('equal', 'datalim')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
save_to_pdf('./plots/umap_neighbor_4_node2vec_clustered.pdf', ax)

In [ ]:
len(list(filter(lambda x: x in user_cat_dict, validation_authors)))

In [ ]:
preds

In [ ]:
vis_df['category'].unique()